# 3D Poisson Equation Solver

This notebook solves the 3D Poisson equation using the recursive exploration policy.

In [1]:
import os
import time
import torch
import pickle
import sympy as sp
import numpy as np
from scipy.optimize import least_squares
from ssde import PDERecursionSolver
from ssde.execute import cython_recursion_execute as ce
from ssde.execute import python_execute as pe
from ssde.program import Program
from ssde.utils import jupyter_logging, cube
from ssde.const import make_const_optimizer
from ssde.pde import function_map
from ssde.task.recursion.recursion import make_regression_metric


np.random.seed(0)
left_bc, right_bc = -1,1
if not os.path.exists('logs'):
    os.makedirs('logs')
solution = sp.sympify("2.5*x1**4 - 1.3*x2**3 + 0.5*x3**2")

def calculate_source(solution):
    ''' Calculate the source term of the PDE '''
    # solution: sympy expr of the solution
    real_params = dict()
    for symbol in sp.preorder_traversal(solution):
        if isinstance(symbol, sp.Symbol):
            exec('%s = sp.Symbol("%s")' % (symbol.name, symbol.name))
            if symbol.name not in real_params:
                real_params[symbol.name] = None
    real_params = sorted(list(real_params.keys()))
    print(f'real_params:{real_params}')
    source = 0
    for i in real_params:
        source += sp.diff(solution, i, 2)
    print(f'source:{source}')
    solution_func = sp.lambdify(real_params, solution, modules='numpy')
    source_func = sp.lambdify(real_params, source, modules='numpy')
    return solution_func, source_func, real_params

def replace_nxexpr(traversals, index=0, new_traversal=None):
    """
    Replace Nxexpr in the traversal with the corresponding expression recursively.

    Parameters
    ----------
    traversals : list
        The list of traversal of the single variable.
    index : int
        The index of current traversal.
    new_traversal : list
        The result of the replacement.
    """
    if new_traversal is None:
        new_traversal = []

    if index + 1 == len(traversals):
        return new_traversal + traversals[index]
    
    current_ls = traversals[index]
    for token in current_ls:
        if token.name != 'Nxexpr':
            new_traversal.append(token)
        else:
            sub_result = replace_nxexpr(traversals, index+1, [])
            new_traversal.extend(sub_result)
    return new_traversal

@jupyter_logging("logs/single_var.log")
def solve_single_var(X_input, y_input, var_index, config_file):
    model = PDERecursionSolver(config_file)
    start_time = time.time()
    config = model.fit(X_input, y_input, start_n_var=var_index)
    print(f'Time used: {time.time()-start_time}')
    traversal = config.program_.traversal
    expr = config.program_.sympy_expr
    print(f'Identified var x{var_index}\'s parametirc expression:')
    print(expr)
    print(f'Identified var x{var_index}\'s traversal:')
    print(traversal)
    return traversal, expr, model

solution_func, source_func, real_params = calculate_source(solution)

real_params:['x1', 'x2', 'x3']
source:30.0*x1**2 - 7.8*x2 + 1.0


## Expression of `x1` 

The differential equation boundary conditions at this point include every face in the 3D square. The boundary condition for the x1 direction will be an infinite number of horizontal lines on the four faces parallel to the x1 axis.

In [2]:
# boundary conditions of x1 direction
n_x1bc = 10
n_x2bc = 2
n_x3bc = 2
X1_bc = np.random.uniform(left_bc, right_bc, (n_x1bc,1))
X2_bc = np.array([left_bc,right_bc])
X3_bc = np.random.uniform(left_bc, right_bc, (n_x3bc,))
X23_bc = np.array([[i,j] for i in X2_bc for j in X3_bc])
y_bc_x1 = []
for i in X23_bc:
    temp = np.tile(i, (n_x1bc, 1))
    X_bc_temp = np.concatenate([X1_bc, temp], axis=1).transpose(1, 0)
    y_bc_x1.append(solution_func(*X_bc_temp).reshape(-1, 1))
X_bc_x1 = X1_bc
y_bc_x1 = np.concatenate(y_bc_x1, axis=1)
print(f'X_bc_x1:{X_bc_x1.shape}, y_bc_x1:{y_bc_x1.shape}')

X_bc_x1:(10, 1), y_bc_x1:(10, 4)


In [3]:
from scipy.signal import convolve2d
# Get the derivative of the solution in the x1 direction as the constraint with finite difference versus the x2 and x3 direction
n_x1bc = 2
n_x23bc = 1001
h = (right_bc - left_bc)/(n_x23bc-1)

X1_bc = np.array([left_bc,right_bc])
X2_bc_all = np.linspace(left_bc, right_bc, n_x23bc)
X3_bc_all = np.linspace(left_bc, right_bc, n_x23bc)
# u_yy + u_zz
solution = solution_func(
    *np.meshgrid(X1_bc, X2_bc_all,  X3_bc_all,indexing='ij'))

laplacian_kernel_high_order = np.array([[0, 0, -1, 0, 0],
                                        [0, 0, 16, 0, 0],
                                        [-1, 16, -60, 16, -1],
                                        [0, 0, 16, 0, 0],
                                        [0, 0, -1, 0, 0]]) / (12 * h**2)

laplacians_high_order = np.zeros((n_x1bc, n_x23bc-4, n_x23bc-4))
for i in range(n_x1bc):
    laplacians_high_order[i,:,:] = convolve2d(solution[i,:,:], laplacian_kernel_high_order, mode='valid')

# Get u_xx
source = source_func(
    *np.meshgrid(X1_bc, X2_bc_all, X3_bc_all,indexing='ij')
)
solution_curtailed = solution[:,2:-2,2:-2]
source_curtailed = source[:,2:-2,2:-2]
u_bc_xx = source_curtailed - laplacians_high_order

# random select 10 points on the left and right face
n_bc_x23 = 10
height = u_bc_xx.shape[1]
width = u_bc_xx.shape[2]
all_index_pairs = np.array(np.meshgrid(np.arange(height), np.arange(width))).T.reshape(-1, 2)
random_indices = np.random.choice(all_index_pairs.shape[0], n_bc_x23, replace=False)
random_point_indices = all_index_pairs[random_indices]
# (n_x1bc, n_bc_x23)
u_xx_bc_x23 = np.array([u_bc_xx[:, index[0], index[1]] for index in random_point_indices]).transpose(1,0)
# (n_x1bc, n_bc_x23)
u_bc_x23 = np.array([solution_curtailed[:, index[0], index[1]] for index in random_point_indices]).transpose(1,0)
# (n_x1bc*2, n_bc_x23)
X_bc_x23 = X1_bc.reshape(-1,1)
y_bc_x23 = np.concatenate([u_xx_bc_x23, u_bc_x23], axis=0)
print(f'X_bc_x23:{X_bc_x23.shape}, y_bc_x23:{y_bc_x23.shape}')
X_input = [X_bc_x23, X_bc_x1]
y_input = [y_bc_x23, y_bc_x1]

X_bc_x23:(2, 1), y_bc_x23:(4, 10)


In [4]:
if os.path.exists('logs/3dpoisson_x1.log'):
    os.remove('logs/3dpoisson_x1.log')
config_file = 'configs/config_poisson_gp.json'
LOG_PATH = 'logs/3dpoisson_x1.log'
x1_traversal, x1_expr, x1_model = solve_single_var(X_input, y_input, 
                                                   1, config_file, log_path=LOG_PATH)
x1_model.save('models/3dpoisson_x1_model')
print('x1 model successfully saved!')
with open('models/3dpoisson_x1_traversal.pkl', 'wb') as f:
    pickle.dump(x1_traversal, f)
print('x1 traversal successfully saved!')

Saved Trainer state to models/3dpoisson_x1_model/trainer.json.
x1 model successfully saved!
x1 traversal successfully saved!


## Expression of `x_2`

In [4]:
# Compute the value of const (actually the label of x2)
n_x1bc = 1
n_x2bc = 20
n_x3bc = 5
# We actually sample the points on the left and right faces, but assemble them into points related to x1, and then optimize nxexpr
X1_bc = np.array([left_bc])
X2_bc = np.linspace(left_bc, right_bc, n_x2bc).reshape(-1,1)
X_bc_x2 = np.concatenate([X1_bc*np.ones((n_x2bc,1)), X2_bc], axis=1)
X3_bc = np.random.uniform(left_bc, right_bc, (n_x3bc,))
# compute the value of the solution at the boundary
X1, X2, X3 = np.meshgrid(X1_bc, X2_bc, X3_bc, indexing='ij')
x1_points = np.stack([X1.ravel(), X2.ravel(), X3.ravel()], axis=-1)
y_bc = solution_func(*x1_points.T).reshape(-1, 1)

In [5]:
with open('models/3dpoisson_x1_traversal.pkl', 'rb') as f:
    x1_traversal = pickle.load(f)

def opti_consts(nxexpr):
    for token in x1_traversal:
        if token.name == 'Nxexpr':
            token.value = nxexpr.reshape(-1,1)
    y_bc_hat = ce(x1_traversal, x1_points)
   
    return (y_bc_hat-y_bc).ravel()

consts = np.ones(x1_points.shape[0])
res = least_squares(opti_consts, consts, method='lm')
y_bc_reshaped = res.x.reshape(n_x1bc, n_x2bc, n_x3bc,1)
y_bc_transposed = y_bc_reshaped.transpose(1,0,2,3)
y_bc_x2 = y_bc_transposed.reshape((n_x2bc, -1))
# test optimization is correct or not
# jin1
consts_real = []
for i in range(n_x3bc):
    consts_real.append(-1.3 * X2_bc**3 + 0.5* X3_bc[i]**2)
consts_real = np.concatenate(consts_real, axis=1)
print('abs error:', np.abs(consts_real-y_bc_x2).mean())

abs error: 1.5128523433993734e-16


In [6]:
X_input = [None, X_bc_x2]
y_input = [None, y_bc_x2]

if os.path.exists('logs/3dpoisson_x2.log'):
    os.remove('logs/3dpoisson_x2.log')
config_file = 'configs/config_poisson_gp.json'
LOG_PATH = 'logs/3dpoisson_x2.log'
x2_traversal, x2_expr, x2_model = solve_single_var(X_input, y_input, 
                                                   2, config_file, log_path=LOG_PATH)
x2_model.save('models/3dpoisson_x2_model')
print('x2 model successfully saved!')
with open('models/3dpoisson_x2_traversal.pkl', 'wb') as f:
    pickle.dump(x2_traversal, f)
    print('x2 traversal successfully saved!')

Saved Trainer state to models/3dpoisson_x2_model/trainer.json.
x2 model successfully saved!
x2 traversal successfully saved!


## Expression of `x_3` 

In [7]:
# Compute the value of const (actually the label of x2)
n_x1bc = 1
n_x2bc = 1
n_x3bc = 20
X1_bc = np.array([left_bc])
X2_bc = np.array([left_bc])
X3_bc = np.random.uniform(left_bc, right_bc, (n_x3bc,))
X1, X2, X3 = np.meshgrid(X1_bc, X2_bc, X3_bc, indexing='ij')

x1_points = np.stack([X1.ravel(), X2.ravel(), X3.ravel()], axis=-1)
X_bc_x3 = x1_points
# compute the value of the solution at the boundary
y_bc = solution_func(*x1_points.T).reshape(-1, 1)

In [8]:
with open('models/3dpoisson_x1_traversal.pkl', 'rb') as f:
    x1_traversal = pickle.load(f)
with open('models/3dpoisson_x2_traversal.pkl', 'rb') as f:
    x2_traversal = pickle.load(f)

        
# replace the nxexpr with x2_traversal
new_traversal = []
for token in x1_traversal:
    if token.name == 'Nxexpr':
        new_traversal.extend(x2_traversal)
    else:
        new_traversal.append(token)


def opti_consts(nxexpr):
    for token in new_traversal:
        if token.name == 'Nxexpr':
            token.value = nxexpr.reshape(-1,1)
    y_bc_hat = ce(new_traversal, x1_points)
    return (y_bc_hat-y_bc).ravel()

consts = np.ones(x1_points.shape[0])
res = least_squares(opti_consts, consts, method='lm')

y_bc_x3 = res.x.reshape(-1,1)
# test optimization is correct or not
# jin1
# consts_real = (0.5* X3_bc**2).reshape(-1,1)
# print('Abs error :', np.abs(consts_real-y_bc_x3).mean())

In [9]:
X_input = [None, X_bc_x3]
y_input = [None, y_bc_x3]

if os.path.exists('logs/3dpoisson_x3.log'):
    os.remove('logs/3dpoisson_x3.log')
config_file = 'configs/config_poisson_gp.json'
LOG_PATH = 'logs/3dpoisson_x3.log'
x3_traversal, x3_expr, x3_model = solve_single_var(X_input, y_input, 
                                                   3, config_file, log_path=LOG_PATH)
x3_model.save('models/3dpoisson_x3_model')
print('x3 model successfully saved!')
with open('models/3dpoisson_x3_traversal.pkl', 'wb') as f:
    pickle.dump(x3_traversal, f)
    print('x3 traversal successfully saved!')

Saved Trainer state to models/3dpoisson_x3_model/trainer.json.
x3 model successfully saved!
x3 traversal successfully saved!


## CFS of 3D Poisson Equation

In [2]:
traversals = []
for i in range(1,4):
    with open(f'models/3dpoisson_x{i}_traversal.pkl', 'rb') as f:
        traversals.append(pickle.load(f))
new_traversal = replace_nxexpr(traversals)

ini_consts = []
for token in new_traversal:
    if token.name == 'const':
        ini_consts.append(token.value)
ini_consts = [torch.tensor(i, requires_grad=True) for i in ini_consts]

In [3]:
# samples for the domain and boundary for refine
n_samples, n_boundary = 10000, 10000
X = np.random.uniform(left_bc, right_bc, (n_samples, 3))
X_bc = cube(left_bc, right_bc, n_boundary)
X_combine = np.concatenate([X, X_bc], axis=0)
X_combine_torch = torch.tensor(X_combine, dtype=torch.float32, requires_grad=True)
y = source_func(*X_combine.T).reshape(-1, 1)
y_bc = solution_func(*X_bc.T).reshape(-1, 1)
y_input = [y, y_bc]
y_input_torch = [torch.tensor(i, dtype=torch.float32, requires_grad=True) for i in y_input]

consts_index = [i for i in range(len(new_traversal)) if new_traversal[i].name == 'const']
metric,_,_ = make_regression_metric("neg_smse_torch", y_input)
def pde_r(consts):
    for i in range(len(consts)):
        new_traversal[consts_index[i]].torch_value = consts[i]
    y = pe(new_traversal, X_combine_torch)
    f = function_map['poisson3d'](y, X_combine_torch)
    y_hat = [f, y[-n_boundary:,0:1]]
    r = metric(y_input_torch,y_hat)
    obj = -r
    return obj

optimized_consts, smse = make_const_optimizer('torch')(pde_r, ini_consts)
for i in range(len(optimized_consts)):
    new_traversal[consts_index[i]].value = optimized_consts[i]
    new_traversal[consts_index[i]].parse_value()
test_p = Program()
test_p.traversal = new_traversal
sym_expr = sp.simplify(sp.N(sp.expand(sp.sympify(test_p.sympy_expr)),4))
print(f'Identified solution: {sym_expr}')


Identified solution: 2.5*x1**4 - 1.3*x2**3 + 0.5*x3**2
